# テキスト分類器の学習
予測までの流れとして、

トークンエンコーディング > トークン埋め込み > エンコーダ > 隠れ層 > 分類ヘッド > 予測

となる。  
※今までのがトークンエンコーディング  

事項からは、DistilBERTで以下両方を検討し、そのトレードオフを検証する  
- 特徴抽出
  - 隠れ状態を特徴として利用し、分類器を学習する。事前学習済みモデルは更新しない。
- ファインチューニング
  - モデル全体をend-to-endで学習する。事前学習済みモデルも同時に更新する。

## 特徴抽出器としてのTransformer

ボディの重みを変えず、分類器のみを学習する。  
GPUなしで素早く学習できるが、浅いモデルとなり、勾配に依存するモデルには不向き。

In [1]:
from transformers import AutoModel
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)
# model_ckptを指定し、指定したデバイスへモデルを読み込んでいる。

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# TIPS: TensorFlowあるいはPyTorchの重みのみリリースされている場合、フレームワーク間の変換が可能
# 以下はPyTorchのみしかリリースされていないxlm-roberta-baseをTensorFlowへ変換する例
cache = """
from transformers import TFAutoModel

tf_model = TFAutoModel.from_pretrained(model_ckpt)
tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base")
tf_xlmr = TFAutoModel.from_pretrained("xlm-roberta-base", from_pt=True)
"""

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [4]:
text = """this is a test"""
inputs = tokenizer(text, return_tensors="pt")
# return_tensors ... PyTorchテンソルに変換する。必須。

print(f"Input tensor shape: {inputs['input_ids'].size()}")
# テンソルは [batch_size, n_tokens] という形状で得られる。

inputs
# inputs.items()

Input tensor shape: torch.Size([1, 6])


{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [5]:
for k, v in inputs.items():
    print(k, v)    

input_ids tensor([[ 101, 2023, 2003, 1037, 3231,  102]])
attention_mask tensor([[1, 1, 1, 1, 1, 1]])


In [6]:
# modelと同じdeviceに、PyTorchテンソルを設置する。
inputs = {k:v.to(device) for k, v in inputs.items()} # items() ... Dict型の要素をキーと値ペアの配列に変換する。順不同。
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231,  102]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [7]:
with torch.no_grad():
    outputs = model(**inputs)
outputs
# torch.no_grad()は勾配の自動計算を無効にしている。 計算のメモリ使用量を減らす事が出来る。

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [8]:
# outputsには 隠れ状態、損失、アテンション等のオブジェクトが含まれる
# 現在のモデルには、最後の隠れ状態である１属性のみを返す
outputs.last_hidden_state.size()
# [batch_size, n_tokens, hidden_dim] という形をしている。
# ∴1バッチ、6個の入力トークン、768次元のベクトルということになる。

torch.Size([1, 6, 768])

In [9]:
# 分類タスクの場合、入力特徴として[CLS]トークンに関連付けられた隠れ状態だけを使うのが一般的
# これは各系列の最初に現れるので、このように抽出可能
outputs.last_hidden_state[:, 0].size() # = [:][0]の意、配列指定の:はすべての行の取得、カンマは次元の区切りを表す。= [:, 0, :]

torch.Size([1, 768])

In [10]:
# これまでの処理をラップ
def extract_hidden_states(batch):
    # モデルの入力をGPU上に配置
    inputs = {k:v.to(device) for k,v in batch.items()
             if k in tokenizer.model_input_names} # 今回の場合、input_idsやattention_maskが含まれていることが条件
    # 最後の隠れ状態を抽出
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state #**inputsは可変長キーワード引数を表す。input_ids, attention_maskなどのキーをまとめて渡せる。
    # [CLS]トークンに対するベクトルを返す
    return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [11]:
from datasets import load_dataset

emotions = load_dataset("dair-ai/emotion")

Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 61283.95 examples/s]


In [12]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [13]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)
emotions_encoded

Map: 100%|██████████| 2000/2000 [00:00<00:00, 35032.67 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [14]:
# input_ids と attension_maskの列をtorch形式に変換
emotions_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [15]:
# 隠れ状態を一度に抽出（予測）
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True) # batch_size=Noneを設定していない場合、デフォルトの1000が使用される
emotions_hidden

Map: 100%|██████████| 2000/2000 [00:02<00:00, 802.69 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
})

In [16]:
emotions_hidden["train"].column_names # hidden_stateが追加されている。

['text', 'label', 'input_ids', 'attention_mask', 'hidden_state']

## 特徴行列の作成
前処理されたデータセットを元に、隠れ状態を入力特徴量として、ラベルをターゲットとして使用する。

In [20]:
import numpy as np

X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
X_train.shape, X_valid.shape

((16000, 768), (2000, 768))

## 学習データセットの可視化
768次元の隠れ状態の可視化は難しいので、UMAPアルゴリズムを使って2次元に射影するとのこと。  

In [22]:
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# 特徴を[0,1]区間にスケール
X_scaled = MinMaxScaler().fit_transform(X_train)
# UMAPの初期化とfit
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
# 2次元埋め込みのDataFrameを作成
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"] = y_train
df_emb.head()

,X,Y,label
0,4.196945,6.377566,0
1,-3.289124,5.459541,0
2,5.344516,3.050781,3
3,-2.531262,3.600675,2
4,-3.565280,3.578017,3


In [ ]:
# TODO_↑コードの理解と次コードでの学習データセットUMAP可視化